In [26]:
# Import required libraries and dependencies
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame and make the index the "coin_id" column.
market_data_df = pd.read_csv("Resources/crypto_market_data.csv", index_col="coin_id")

# Display sample data
market_data_df.head(10)

In [ ]:
# Generate summary statistics
market_data_df.describe()

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_data_scaled = StandardScaler().fit_transform(market_data_df)
market_data_scaled

In [ ]:
# Create a DataFrame with the scaled data
scaled_df = pd.DataFrame(market_data_scaled, columns=market_data_df.columns)

# Copy the crypto names from the original data
coinid = market_data_df.index

# Set the coinid column as index
scaled_df.set_index(coinid, inplace=True)

# Display sample data
scaled_df.head(5)

### Find the Best Value for k Using the Original Scaled DataFrame.

In [ ]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k = list(range(1,11))

# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using the scaled DataFrame
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i, n_init='auto', random_state=0)
    k_model.fit(scaled_df)
    inertia.append(k_model.inertia_)

# Create a dictionary with the data to plot the Elbow curve
elbow_data = {'k':k, 'inertia':inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

# Display the DataFrame
df_elbow

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow.plot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

#### Answer the following question: 
**Question:** What is the best value for `k`?

**Answer:** 4

### Cluster Cryptocurrencies with K-means Using the Original Scaled Data.

In [33]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, n_init='auto', random_state=0)

In [ ]:
# Fit the K-Means model using the scaled data
model.fit(scaled_df)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
k_4 = model.predict(scaled_df)

# View the resulting array of cluster values.
k_4

In [36]:
# Create a copy of the DataFrame
crypto_predictions = scaled_df.copy()

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
crypto_predictions['crypto_cluster'] = k_4

# Display sample data
crypto_predictions.head(5)

In [ ]:
# Create a scatter plot using Pandas plot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Use "rainbow" for the color to better visualize the data.
crypto_predictions.plot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    c="crypto_cluster",
    colormap='winter')

### Optimize Clusters with Principal Component Analysis.

In [39]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` on the original scaled DataFrame to reduce to three principal components.
market_pca = pca.fit_transform(scaled_df)

# View the first five rows of the DataFrame. 
market_pca[:5]

In [ ]:
# Retrieve the explained variance to determine how much information  can be attributed to each principal component.
pca.explained_variance_ratio_

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total variance exaplined by the selected components is approximately: 89.49%

In [ ]:
# Create a new DataFrame with the PCA data.
# Note: The code for this step is provided for you

# Creating a DataFrame with the PCA data
pca_df = pd.DataFrame(market_pca, columns=['PCA1', 'PCA2', 'PCA3'])

# Copy the crypto names from the original data
coinid = market_data_df.index

# Set the coinid column as index
pca_df.set_index(coinid, inplace=True)

# Display sample data
pca_df.head(10)

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k = list(range(1,11))

# Create an empty list to store the inertia values
pca_inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using PCA DataFrame.
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i, n_init='auto', random_state=0)
    k_model.fit(pca_df)
    pca_inertia.append(k_model.inertia_)

# Create a dictionary with the data to plot the Elbow curve
pca_elbow_data = {'k':k, 'inertia': pca_inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_pca_elbow = pd.DataFrame(pca_elbow_data)

# Display the DataFrame
df_pca_elbow

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_pca_elbow.plot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

#### Answer the following questions: 
* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** It does not differ from the original data


### Cluster Cryptocurrencies with K-means Using the PCA Data

In [45]:
# Initialize the K-Means model using the best value for k
k_model_pca_data = KMeans(n_clusters=4, n_init='auto', random_state=0)

In [ ]:
# Fit the K-Means model using the PCA data
k_model_pca_data.fit(pca_df)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
k_4_pca_predictions = k_model_pca_data.predict(pca_df)

# Print the resulting array of cluster values.
k_4_pca_predictions

In [ ]:
# Create a copy of the DataFrame with the PCA data
pca_crypto_predictions = pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
pca_crypto_predictions['crypto_cluster'] = k_4_pca_predictions

# Display sample data
pca_crypto_predictions.head()

In [ ]:
# Create a scatter plot using hvPlot by setting `x="PCA1"` and `y="PCA2"`. 
pca_crypto_predictions.plot.scatter(
    x="PCA1",
    y="PCA2",
    c="crypto_cluster",
    colormap='winter'
)

### Determine the Weights of Each Feature on each Principal Component

In [ ]:
# Use the columns from the original scaled DataFrame as the index.
pca_component_weights = pd.DataFrame(pca.components_.T, columns=['PCA1', 'PCA2', 'PCA3'], index=scaled_df.columns)
pca_component_weights

#### Answer the following question: 

* **Question:** Which features have the strongest positive or negative influence on each component? 
 
* **Answer:** 
- "price_change_percentage_200d", and "price_change_percentage_1y" have the strongest positive influence on PCA1. "price_change_percentage_24h" has the strongest negative influence on PCA1.
- "price_change_percentage_30d" and "price_change_percentage_14d" have the strongest positive influence on PCA2.
- "price_change_percentage_7d" has the strongest positive influence on PCA2.